# Require Input
1. [Fill in File Input](#Load-in-Data)
2. [Right or Left Hand](#Right-or-Left-Hand)
3. [Output ShotCSV](#Output-ShotCSV)
4. [Change to atNet (by Player) from tagger later](#At-Net)
5. [EDA and get Stats Here](#EDA)
6. [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
7. [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)

# Need to change soon
- atNetPlayer1, atNetPlayer2 are being aggregated by if hitting a volley. Will be replaced by tagger values for those
- pointWonBy is being calculated, replaced by tagger soon
- player1Hand and player2Hand are being manually inputted

# Add
- More stats on approach shots
- Pressure Point stats, like what did players did on 30-30 points onwards
- Inside Out/Inside In Stats once we get matches with accurate side info
- handling of rows where there is error on video side and the point is cut short

Notes:
- Keep empty values None Type, they will be converted to empty string "" right before outputting to Point CSV Only for Upload
- Having 1 does not mean other value is boolean 0, it is None

# Post Process
- Adds Point Number (Comment out if already there)
- Forward Fills these columns'gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'
   - Add this when in tagger: player1Hand, player2Hand 
- Adds Returner Name
- Adds Shot Hit By for each Shot
- Adds isInsidein/isInsideOut (booleans 0/1) Columns

In [964]:
import pandas as pd
import numpy as np

## Load in Data

In [965]:
# Put your file name here
shot_data = pd.read_csv("./oldtagger/TianFangRan_vs_SabinaZeynalova_Merged.csv")



# Fill in Player Name

In [966]:
if shot_data.at[0, 'player1Name'] == None:
    print("player1Name was blank. Filling with 'fillInplayer1Name'")
    shot_data.at[0, 'player1Name'] = 'fillInplayer1Name'
else:
    print("player1Name is not blank. Current value:", shot_data.at[0, 'player1Name'])

# Check if player2Name is blank, if so, fill it with 'fillInplayer2Name'
if shot_data.at[0, 'player2Name'] == None:
    print("player2Name was blank. Filling with 'fillInplayer2Name'")
    shot_data.at[0, 'player2Name'] = 'fillInplayer2Name'
else:
    print("player2Name is not blank. Current value:", shot_data.at[0, 'player2Name'])


player1Name is not blank. Current value: Tian Fang Ran
player2Name is not blank. Current value: Sabina Zeynalova


- add side and pointNumber Columns

In [967]:
# # IF NO SIDE Column (Old tagger)
# deuce_values = ['0-0', '15-15', '40-15', '30-0', '0-30', '15-40', '40-40 (Deuce)', '30-30']
# shot_data['side'] = shot_data.apply(lambda row: 'Deuce' if str(row['pointScore']) in deuce_values and row['isPointStart'] == 1 else 'Ad' if str(row['pointScore']) is not None and row['isPointStart'] == 1 else None, axis=1)


In [968]:
 # Check if both conditions passed
if 'pointNumber' in shot_data.columns and not shot_data['pointNumber'].isnull().any() and shot_data['pointNumber'].is_monotonic_increasing:
    print("All conditions passed: 'pointNumber' column exists for every row, is not empty, and is in increasing order.")


# Check if pointNumber exists for every row and is not empty
if 'pointNumber' not in shot_data.columns or shot_data['pointNumber'].isnull().any():
    # Execute the script to calculate pointNumber
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
    
    print("Condition 1 failed: 'pointNumber' column does not exist for every row or contains empty values.")
    
# Check if pointNumber is in increasing order
if not shot_data['pointNumber'].is_monotonic_increasing:
    # Print the condition failure if the 'pointNumber' column is not in increasing order
    print("Condition 2 failed: 'pointNumber' column is not in increasing order.")
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
   

Condition 1 failed: 'pointNumber' column does not exist for every row or contains empty values.


In [969]:
missing_rows = shot_data[shot_data['pointNumber'].isnull()]
print(f"Count of rows missing 'pointNumber': {len(missing_rows)}")
# print(missing_rows)

Count of rows missing 'pointNumber': 0


- Check for NA's (Revise Data if needed)
- shotInRally
- pointScore
- gameScore
- setScore
- side
- pointStartTime when isPointStart = 1
- pointEndTime when sPointEnd = 1

In [970]:
# Count empty strings in each column
empty_string_counts = (shot_data == "").sum()

# Filter out columns with zero empty strings
non_zero_counts = empty_string_counts[empty_string_counts > 0]

# Display the counts
print("Count of empty strings in each column that includes at least one:")
print(non_zero_counts)
# Count NaN or empty values in specified columns
na_counts = shot_data[['shotInRally', 'pointScore', 'gameScore', 'setScore', 'side', 'player1Name', 'player2Name','serverName']].isna().sum()

# Display the counts
print(f"\nCout of NA's in these columns\n{na_counts}")

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
na_count = shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].isna().sum()

# Display the count
print("\nCount of NaN values in 'pointScore' when 'gameScore' is '6-6':", na_count)

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
side_na_count = shot_data.loc[shot_data['isPointStart'] == 1, 'side'].isna().sum()

# Display the count
print("\nCount of NaN values for side on Point Start:", side_na_count)



Count of empty strings in each column that includes at least one:
Series([], dtype: int64)

Cout of NA's in these columns
shotInRally      0
pointScore      91
gameScore        0
setScore         0
side           472
player1Name    818
player2Name    818
serverName       0
dtype: int64

Count of NaN values in 'pointScore' when 'gameScore' is '6-6': 91

Count of NaN values for side on Point Start: 0


In [971]:
empty_shot_rows = shot_data[shot_data['shotInRally'].isnull()]

# Iterate over the index of empty_shot_rows
for index in empty_shot_rows.index:
    # Get the row with empty shotInRally
    empty_row = shot_data.loc[index]
    
    # Get the row above it
    if index - 1 >= 0:
        above_row = shot_data.loc[index - 1]
    else:
        above_row = None
    
    # Get the row below it
    if index + 1 < len(shot_data):
        below_row = shot_data.loc[index + 1]
    else:
        below_row = None
    
    # Print the rows
    print("Empty row:")
    print(empty_row)
    print("Row above:")
    print(above_row)
    print("Row below:")
    print(below_row)
    print("====================")


In [972]:
# Find rows where isPointStart is True, but missing pointStartTime
missing_start_time = shot_data[((shot_data['isPointStart'] == 1) & shot_data['pointStartTime'].isna())]
print(f"Count of rows where PointStartTime is missing: {len(missing_start_time)}")


print(f"These are rows where PointStartTime is missing \n{missing_start_time}")

Count of rows where PointStartTime is missing: 3
These are rows where PointStartTime is missing 
    pointScore gameScore setScore tiebreakScore  isPointStart  pointStartTime  \
586      40-15       1-0      1-0           NaN           1.0             NaN   
616        0-0       3-1      1-0           NaN           1.0             NaN   
774      30-15       5-3      1-0           NaN           1.0             NaN   

     isPointEnd  pointEndTime  pointNumber  isBreakPoint  ...  Date Division  \
586         NaN           NaN           92           NaN  ...   NaN      NaN   
616         NaN           NaN          104           NaN  ...   NaN      NaN   
774         NaN           NaN          134           NaN  ...   NaN      NaN   

    Event  matchDetails opponentTeam  player1Name  player2Name  Round Surface  \
586   NaN           NaN          NaN          NaN          NaN    NaN     NaN   
616   NaN           NaN          NaN          NaN          NaN    NaN     NaN   
774   NaN     

In [973]:
# Find rows where isPointEnd is True, but missing pointEndTime
missingEndTime = shot_data[((shot_data['isPointEnd'] == 1) & shot_data['pointEndTime'].isna())]
print(f"Count of rows where PointEndTime is missing: {len(missingEndTime)}")

# Display the missing rows 
print(f"These are rows where PointEndTime is missing \n{missingEndTime}")

Count of rows where PointEndTime is missing: 6
These are rows where PointEndTime is missing 
    pointScore gameScore setScore tiebreakScore  isPointStart  pointStartTime  \
361       15-0       5-4      0-0           NaN           NaN             NaN   
377      40-15       5-4      0-0           NaN           NaN             NaN   
378      40-30       5-4      0-0           NaN           1.0       3143732.0   
588      40-15       1-0      1-0           NaN           NaN             NaN   
621        0-0       3-1      1-0           NaN           NaN             NaN   
788      30-15       5-3      1-0           NaN           NaN             NaN   

     isPointEnd  pointEndTime  pointNumber  isBreakPoint  ...  Date Division  \
361         1.0           NaN           52           NaN  ...   NaN      NaN   
377         1.0           NaN           55           NaN  ...   NaN      NaN   
378         1.0           NaN           56           NaN  ...   NaN      NaN   
588         1.0    

- Make Jan-00 back into 1-0 for Game/Set Score

In [974]:
# Make Scores Strings not Date Time
columns_to_convert = ['gameScore', 'setScore'] #if no tiebreakScore
# columns_to_convert = ['gameScore', 'setScore', 'tiebreakScore']
shot_data[columns_to_convert] = shot_data[columns_to_convert].astype(object)

In [975]:
import re

# Define a mapping for month abbreviations
month_mapping = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

# Function to convert string like 'Jan-00' to '1-0'
def convert_score_string(score_str):
    # Check if the string has a month abbreviation and a year ending with '00'
    if re.match(r'^\d{1,2}-[A-Za-z]{3}$', score_str):
        # Extract year and month abbreviation
        year, month = score_str.split('-')

        # Remove leading zeros from the year
        year = str(int(year))

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Concatenate the parts to form the transformed string
        transformed_str = f'{year}-{month_number}'

        return transformed_str

    # Check if the string has a month abbreviation and a year with leading '0's
    elif re.match(r'^[A-Za-z]{3}-\d{1,2}$', score_str):
        # Extract month abbreviation and year
        month, year = score_str.split('-')

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Remove leading zeros from the year
        year = str(int(year))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month_number}-{year}'

        return transformed_str

    return score_str

# Apply the conversion function to the relevant columns in shot_data
shot_data['gameScore'] = shot_data['gameScore'].apply(convert_score_string)
shot_data['setScore'] = shot_data['setScore'].apply(convert_score_string)
# shot_data['tiebreakerScore'] = shot_data['tiebreakerScore'].apply(convert_score_string)


In [976]:
# Assuming shot_data is your DataFrame
unique_set_scores = shot_data['setScore'].unique()
unique_game_scores = shot_data['gameScore'].unique()

# Print unique values
print("Unique Set Scores:", unique_set_scores)
print("Unique Game Scores:", unique_game_scores)


Unique Set Scores: ['0-0' '1-0']
Unique Game Scores: ['0-0' '1-0' '2-0' '2-1' '3-1' '3-2' '3-3' '4-3' '5-3' '5-4' '5-5' '6-5'
 '6-6' '4-1' '4-2']


In [977]:
# Create new columns
import pandas as pd

# Assuming shot_data is your DataFrame

# Define functions to extract shotFhBh and shotDirection from shotType
def extract_shotFhBh(x):
    if 'Forehand' in str(x):
        return 'Forehand'
    elif 'Backhand' in str(x):
        return 'Backhand'
    else:
        return None

def extract_shotDirection(x):
    if 'Crosscourt' in str(x):
        return 'Crosscourt'
    elif 'Down the Line' in str(x):
        return 'Down the Line'
    else:
        return None

# Apply the functions to create new columns, but only when shotFhBh and shotDirection are empty
shot_data['shotFhBh'] = shot_data['shotFhBh'].fillna(shot_data['shotType'].apply(extract_shotFhBh))
shot_data['shotDirection'] = shot_data['shotDirection'].fillna(shot_data['shotType'].apply(extract_shotDirection))

# Similarly, apply the functions to returnData, but only when shotFhBh and shotDirection are still empty
shot_data.loc[pd.notna(shot_data['returnData']) & pd.isna(shot_data['shotFhBh']), 'shotFhBh'] = shot_data.loc[pd.notna(shot_data['returnData']), 'returnData'].apply(extract_shotFhBh)
shot_data.loc[pd.notna(shot_data['returnData']) & pd.isna(shot_data['shotDirection']), 'shotDirection'] = shot_data.loc[pd.notna(shot_data['returnData']), 'returnData'].apply(extract_shotDirection)


# Display the resulting DataFrame
print(shot_data[['shotType', 'returnData', 'shotFhBh', 'shotDirection']].tail(15))

     shotType returnData  shotFhBh  shotDirection
804       NaN        NaN      None           None
805  Forehand        NaN  Forehand  Down the Line
806       NaN        NaN      None           None
807  Backhand        NaN  Backhand     Crosscourt
808  Forehand        NaN  Forehand     Crosscourt
809  Backhand        NaN  Backhand  Down the Line
810  Forehand        NaN  Forehand     Crosscourt
811       NaN        NaN      None           None
812  Forehand        NaN  Forehand  Down the Line
813  Backhand        NaN  Backhand     Crosscourt
814  Backhand        NaN  Backhand     Crosscourt
815       NaN        NaN      None           None
816  Backhand        NaN  Backhand     Crosscourt
817  Backhand        NaN  Backhand  Down the Line
818  Forehand        NaN  Forehand     Crosscourt


- Fill in any NA for ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name'] by forward filling
- Example: (1, NA, NA) -> (1, 1, 1)

# Add More Columns
- ReturnerName 
- shotHitBy
- isInsideOut
- isInsideIN


In [978]:
# Get Returner Name for Shot DF

# Extract player1Name and player2Name from the first row of shot_data
first_row_shot_df = shot_data.iloc[0]
player1_name = first_row_shot_df['player1Name']
player2_name = first_row_shot_df['player2Name']

# # Can Also Manually inpute Player 1 and 2 name
# player1_name = "Kimmi Hance"
# player2_name = "Malaika Rapolu"

def get_returner_name(server_name):
    return player2_name if server_name == player1_name else player1_name

# Add 'Returner Name' column to point_df using the function
shot_data['returnerName'] = shot_data['serverName'].apply(get_returner_name)

print(f"Player 1 = {player1_name}, Player 2 = {player2_name}")
# print(shot_data[['serverName','returnerName']])

Player 1 = Tian Fang Ran, Player 2 = Sabina Zeynalova


In [979]:
# Add Shot Hit By for each Shot

shot_data['shotHitBy'] = shot_data.apply(lambda row: row['serverName'] if row['shotInRally'] % 2 == 1 else row['returnerName'], axis=1)
# print(shot_data[['serverName', 'shotHitBy','pointNumber', 'shotInRally']].head(20))

# Right or Left Hand

In [980]:
# Change this with whatever is player1Hand and player2Hand first Column Values
fill_in_player1hand = "Right"
fill_in_player2hand = "Right"

# Find the first non-null value in player1Hand column
player1hand = shot_data['player1Hand'].dropna().iloc[0] if 'player1Hand' in shot_data.columns else fill_in_player1hand
player2hand = shot_data['player2Hand'].dropna().iloc[0] if 'player2Hand' in shot_data.columns else fill_in_player2hand

# Display the updated values
print("player1Hand:", player1hand)
print("player2Hand:", player2hand)

shot_data['player1Hand'] = player1hand
shot_data['player2Hand'] = player2hand

player1Hand: Right
player2Hand: Right


In [981]:
# Add columns for isInsideOut and isInsideIn, initially set to 0
shot_data['isInsideOut'] = None
shot_data['isInsideIn'] = None

# Iterate through rows
for index, row in shot_data.iterrows():
    shotHitBy = row['shotHitBy']
    player_hand = player1hand if shotHitBy == row['player1Name'] else player2hand
    
    if player_hand == "Right":
        if row['side'] == "Deuce" and row['shotType'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotType'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotType'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Ad" and row['shotType'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
    elif player_hand == "Left":
        if row['side'] == "Ad" and row['shotType'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotType'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotType'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Deuce" and row['shotType'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1

In [982]:
# Add the Ace column
shot_data['isAce'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] == 1 or row['secondServeIn'] == 1): # either first or second serve went in
                shot_data.at[index, 'isAce'] = 1

In [983]:
# Add the DoubleFault column
shot_data['isDoubleFault'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] != 1 and row['secondServeIn'] != 1): # either first or second serve went in
                shot_data.at[index, 'isDoubleFault'] = 1

In [984]:
# Add the 'pointWonBy' column
shot_data['pointWonBy'] = None

# Add the 'lastShotError' column
shot_data['lastShotError'] = 0

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if row['isAce'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['serverName']
            elif row['isDoubleFault'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['returnerName']

                
        elif row['shotInRally'] != 1:
            if row['isErrorWideR'] == 1 or row['isErrorWideL'] == 1 or row['isErrorNet'] == 1 or row['isErrorLong'] == 1: # if error
                shot_data.at[index, 'lastShotError'] = 1
                
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
        
            elif row['isWinner'] == 1:
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']


In [985]:
shot_data['serverName'].unique()

array(['Tian Fang Ran', 'Sabina Zeynalova'], dtype=object)

In [986]:
# Forward Fill GameScore, SetScore, tiebreakScore, serverName, player1Name, player2Name

columns_to_fill = ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name', 'player1Hand', 'player2Hand'] 
# columns_to_fill = ['gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'] 
# Add player1Hand, player2Hand when it is in tagger

for column in columns_to_fill:
    shot_data[column].replace(['', 'na'], pd.NaT, inplace=True)
    shot_data[column] = shot_data[column].ffill()

In [987]:
print(shot_data[(shot_data['isPointEnd'] == 1) & (shot_data['pointWonBy'].isnull())][['pointScore', 'gameScore','setScore', 'lastShotError', 'isWinner', 'isErrorWideR', 'isErrorWideL',
       'isErrorNet', 'isErrorLong', 'pointWonBy', 'serverName', 'shotInRally']])


    pointScore gameScore setScore  lastShotError  isWinner  isErrorWideR  \
60        0-15       2-0      0-0              0       NaN           NaN   
358        0-0       5-4      0-0              0       NaN           NaN   
517        NaN       6-6      0-0              0       NaN           NaN   

     isErrorWideL  isErrorNet  isErrorLong pointWonBy        serverName  \
60            NaN         NaN          NaN       None     Tian Fang Ran   
358           NaN         NaN          NaN       None  Sabina Zeynalova   
517           NaN         NaN          NaN       None     Tian Fang Ran   

     shotInRally  
60             2  
358            3  
517            8  


In [988]:


# Count where isPointStart = 1
count_isPointStart = (shot_data['isPointStart'] == 1).sum()

# Count where isPointEnd = 1
count_isPointEnd = (shot_data['isPointEnd'] == 1).sum()

print("Total count where isPointStart = 1:", count_isPointStart)
print("Total count where isPointEnd = 1:", count_isPointEnd)



# Initialize variables to keep track of the state
has_start = False
overlapping_ends = []

# Iterate through the DataFrame
for i in range(len(shot_data)):
    if shot_data['isPointStart'][i] == 1:
        has_start = True
    elif shot_data['isPointEnd'][i] == 1:
        if not has_start:
            overlapping_ends.append(i)
        else:
            has_start = False

# Display the rows where isPointEnd doesn't have a corresponding isPointStart
print("Rows where isPointEnd doesn't have a corresponding isPointStart:")
print(shot_data.iloc[overlapping_ends])

# Initialize variables to keep track of the state
start_indices = []
consecutive_starts = []

# Iterate through the DataFrame
for i in range(len(shot_data) - 1):  # Iterate until the second last row
    if shot_data['isPointStart'][i] == 1 and shot_data['isPointEnd'][i] != 1:
        # Check if the next row also has isPointStart without a corresponding isPointEnd
        if shot_data['isPointStart'][i + 1] == 1 and shot_data['isPointEnd'][i + 1] != 1:
            consecutive_starts.append(i + 1)

# Display the rows where isPointStart may double up without a corresponding isPointEnd
print("Rows where isPointStart may double up without a corresponding isPointEnd:")
print(shot_data.iloc[consecutive_starts])


Total count where isPointStart = 1: 141
Total count where isPointEnd = 1: 141
Rows where isPointEnd doesn't have a corresponding isPointStart:
Empty DataFrame
Columns: [pointScore, gameScore, setScore, tiebreakScore, isPointStart, pointStartTime, isPointEnd, pointEndTime, pointNumber, isBreakPoint, shotInRally, side, serverName, firstServeIn, firstServeZone, firstServeXCoord, firstServeYCoord, secondServeIn, secondServeZone, secondServeXCoord, secondServeYCoord, isAce, returnData, returnContactX, returnContactY, returnPlacementX, returnPlacementY, shotContactX, shotContactY, shotDirection, shotFhBh, shotType, isSlice, isVolley, isOverhead, isApproach, isDropshot, shotLocationX, shotLocationY, isWinner, isErrorWideR, isErrorWideL, isErrorNet, isErrorLong, clientTeam, Date, Division, Event, matchDetails, opponentTeam, player1Name, player2Name, Round, Surface, Notes, returnerName, shotHitBy, player1Hand, player2Hand, isInsideOut, isInsideIn, isDoubleFault, pointWonBy, lastShotError]
Index

In [989]:
player1Name = shot_data.iloc[0]['player1Name']

# Filter shot_data based on the conditions
approach_data_player1 = shot_data[(shot_data['isApproach'] == 1) & (shot_data['shotHitBy'] == player1Name)]

# Count the distinct pointNumbers
distinct_point_numbers = approach_data_player1['pointNumber'].nunique()

# Print the result
print("Number of distinct pointNumbers including isApproach = 1 and shotHitBy = player1Name:", distinct_point_numbers)

print(approach_data_player1)


Number of distinct pointNumbers including isApproach = 1 and shotHitBy = player1Name: 8
    pointScore gameScore setScore tiebreakScore  isPointStart  pointStartTime  \
163        0-0       3-2      0-0           NaN           NaN             NaN   
219      15-40       3-3      0-0           NaN           NaN             NaN   
543        NaN       6-6      0-0           6-6           NaN             NaN   
588      40-15       1-0      1-0           NaN           NaN             NaN   
621        0-0       3-1      1-0           NaN           NaN             NaN   
642      40-40       3-1      1-0           NaN           NaN             NaN   
734       30-0       4-3      1-0           NaN           NaN             NaN   
737       40-0       4-3      1-0           NaN           1.0       6767748.0   

     isPointEnd  pointEndTime  pointNumber  isBreakPoint  ...  Notes  \
163         NaN           NaN           27           NaN  ...    NaN   
219         NaN           NaN         

In [990]:
shot_data.replace("", None, inplace=True)

# Output ShotCSV

In [991]:
# # Ouput Improved Shot Csv HERE
# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'{player1NameNoSpace}_{player2NameNoSpace}_Shot.csv', index=False)

In [992]:
shot_data['pointNumber'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141])

# Below is for Point CSV

# Create Point DF

In [993]:
# Creating point_df (with only 1 row for each pointNumber)
point_df = shot_data.drop_duplicates(subset='pointNumber')[['pointNumber']]

In [994]:
point_df.shape

(141, 1)

In [995]:
# Extract the first value of player1Name and player2Name from shot_data
first_player1Name = shot_data['player1Name'].iloc[0]
first_player2Name = shot_data['player2Name'].iloc[0]

# Fill in the first value into all rows of point_df['player1Name'] and point_df['player2Name']
point_df['player1Name'] = first_player1Name
point_df['player2Name'] = first_player2Name

# Add Scores


In [996]:

point_df['pointScore'] = shot_data.groupby('pointNumber')['pointScore'].first().values
point_df['gameScore'] = shot_data.groupby('pointNumber')['gameScore'].first().values
point_df['tiebreakScore'] = shot_data.groupby('pointNumber')['tiebreakScore'].first().values

# Add Side

In [997]:
# Group shot_data by 'pointNumber' and get the first 'side' value for each group
side_values = shot_data.groupby('pointNumber')['side'].first().reset_index()

point_df['side'] = side_values['side'].values

In [998]:
# Adds Server and Returner Names and pointScore

point_df['serverName'] = shot_data.groupby('pointNumber')['serverName'].first().values
point_df['returnerName'] = shot_data.groupby('pointNumber')['returnerName'].first().values


In [999]:
# Add Start and End times per point

for index, row in shot_data.iterrows():
    point_number = row['pointNumber']
    
    if row['isPointStart'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'Position'] = row['pointStartTime']
    if row['isPointEnd'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'pointEndPosition'] = row['pointEndTime']

# Add Duration
point_df['Duration'] = point_df['pointEndPosition'] - point_df['Position']

# Add Rally Column
- rallyCount (x amount)
- rallyCountFreq (1-4,5-8,9-12,13+)

In [1000]:
# Find the highest shotInRally for each pointNumber in shot_data
max_rally_per_point = shot_data.groupby('pointNumber')['shotInRally'].max().reset_index()
point_df['rallyCount'] = list(max_rally_per_point['shotInRally'])

# Add 'rallyCountFreq' column based on specified conditions
point_df['rallyCountFreq'] = point_df['rallyCount'].apply(lambda x: '1 - 4' if 1 <= x <= 4 else ('5 - 8' if 5 <= x <= 8 else ('9 - 12' if 9 <= x <= 12 else ('13 +' if x >= 13 else 'Error'))))


# Serve 
- firstServeIn
- secondServeIn
- serveResult
- serveInPlacement

In [1001]:
# Add firstServeIn and secondServeIn

# Add firstServeIn and secondServeIn columns
point_df['firstServeIn'] = 0
point_df['secondServeIn'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'firstServeIn'] = 1
    
    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'secondServeIn'] = 1

In [1002]:
# Add serveResult and serveInPlacement

point_df['serveResult'] = None
point_df['serveInPlacement'] = None

for point_number in shot_data['pointNumber'].unique():
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'serveResult'] = '1st Serve In'
        point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['firstServeZone']
    elif any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'serveResult'] = '2nd Serve In'
        point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']    
    else: 
        point_df.loc[point_df['pointNumber'] == point_number, 'serveResult'] = 'Double Fault'
        point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = np.nan
        
    point_df.loc[point_df['pointNumber'] == point_number, 'firstServeZone'] = shot_data['firstServeZone']
    point_df.loc[point_df['pointNumber'] == point_number, 'secondServeZone'] = shot_data['secondServeZone']
    
# Display the result
print(point_df['serveInPlacement'].unique())


['Wide' nan 'T' 'Body']


In [1003]:
serve_zones = shot_data.loc[shot_data['shotInRally'] == 1, ['pointNumber', 'firstServeZone', 'secondServeZone', 'firstServeIn', 'secondServeIn']].drop_duplicates()


point_df['firstServeZone'] = shot_data.groupby('pointNumber')['firstServeZone'].first().values
point_df['secondServeZone'] = shot_data.groupby('pointNumber')['secondServeZone'].first().values



In [1004]:
double_fault_rows = point_df[point_df['serveResult'] == 'Double Fault']
# print(double_fault_rows)

# Ace

In [1005]:
point_df['isAce'] = ((point_df['rallyCount'] == 1) & ((point_df['serveResult'] != "Double Fault")))

# Display the resulting DataFrame
print(point_df[['pointNumber', 'rallyCount', 'isAce']].head(14))

    pointNumber  rallyCount  isAce
0             1           3  False
3             2           2  False
5             3           2  False
7             4           2  False
9             5          11  False
20            6           2  False
22            7           2  False
24            8           3  False
27            9          17  False
44           10           2  False
46           11           8  False
54           12           5  False
59           13           2  False
61           14           4  False


# Return

In [1006]:
# Assuming point_df is your DataFrame

# Set the initial values of 'returnDirection' and 'returnHand' columns to None
point_df['returnDirection'] = None
point_df['returnFhBh'] = None

# Iterate through pointNumber in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if shotInRally == 2 exists for the given pointNumber
    if 2 in shot_data.loc[shot_data['pointNumber'] == point_number, 'shotInRally'].values:
        # Get the information from the corresponding row
        row_with_return_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['shotInRally'] == 2)].iloc[0]

        # Assign values to 'returnDirection' and 'returnHand' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'returnDirection'] = row_with_return_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'returnFhBh'] = row_with_return_info['shotFhBh']

# Display the modified DataFrame
print(point_df.head(10))


    pointNumber    player1Name       player2Name pointScore gameScore  \
0             1  Tian Fang Ran  Sabina Zeynalova        0-0       0-0   
3             2  Tian Fang Ran  Sabina Zeynalova       0-15       0-0   
5             3  Tian Fang Ran  Sabina Zeynalova      15-15       0-0   
7             4  Tian Fang Ran  Sabina Zeynalova      30-15       0-0   
9             5  Tian Fang Ran  Sabina Zeynalova      40-15       0-0   
20            6  Tian Fang Ran  Sabina Zeynalova      40-30       0-0   
22            7  Tian Fang Ran  Sabina Zeynalova        0-0       1-0   
24            8  Tian Fang Ran  Sabina Zeynalova       15-0       1-0   
27            9  Tian Fang Ran  Sabina Zeynalova      15-15       1-0   
44           10  Tian Fang Ran  Sabina Zeynalova      15-30       1-0   

   tiebreakScore   side        serverName      returnerName  Position  ...  \
0           None  Deuce     Tian Fang Ran  Sabina Zeynalova   41482.0  ...   
3           None     Ad     Tian Fang Ra

# Add Error Column

In [1007]:
# Create an empty DataFrame to store the results
error_results = pd.DataFrame(columns=['errorType', 'pointNumber'])

# Iterate through entire shot_data
for index, row in shot_data.iterrows():
    pointNumber = row['pointNumber']
    point_error_value = None
    
    if row['isErrorWideR'] == 1:
        point_error_value = 'Wide Right'
    elif row['isErrorWideL'] == 1:
        point_error_value = 'Wide Left'
    elif 'isErrorNet' in row and row['isErrorNet'] == 1:
        point_error_value = 'Net'
    elif row['isErrorLong'] == 1:
        point_error_value = 'Long'
    

    # If an error is found, append the result to the error_results DataFrame
    if point_error_value is not None:
        error_results = pd.concat([error_results, pd.DataFrame({'pointNumber': [pointNumber], 'errorType': [point_error_value]})], ignore_index=True)


# Drop duplicates based on 'pointNumber'
error_results = error_results.drop_duplicates(subset=['pointNumber'])

In [1008]:
# Create a dictionary mapping 'pointNumber' to 'errorType' in error_results
error_type_mapping = dict(zip(error_results['pointNumber'], error_results['errorType']))

# Create 'errorType' column in point_df based on the mapping
point_df['errorType'] = point_df['pointNumber'].map(error_type_mapping)

point_df = point_df.replace({np.nan: None})

# Return Stuff

In [1009]:
def get_return_error(row):
    if row['rallyCount'] == 2:
        return row['errorType']
    else:
        return None

point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']  

# Apply the functions to create the new columns
point_df['returnError'] = point_df.apply(get_return_error, axis=1)


In [1010]:
# print(point_df["returnError"].unique())
# print(point_df[['rallyCount','returnDirection','returnFhBh']])

# Last Shot

In [1011]:

point_df['lastShotDirection'] = None
point_df['lastShotFhBh'] = None
point_df['lastShotHitBy'] = None  
point_df['lastShotResult'] = None  

# Iterate through unique pointNumbers in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if isPointEnd == 1 exists for the given pointNumber
    if 1 in shot_data.loc[shot_data['pointNumber'] == point_number, 'isPointEnd'].values:
        # Get the information from the corresponding row
        row_with_lastshot_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['isPointEnd'] == 1)].iloc[0]

        # Assign values to 'lastShotDirection' and 'lastShotFhBh' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotDirection'] = row_with_lastshot_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotFhBh'] = row_with_lastshot_info['shotFhBh']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotHitBy'] = row_with_lastshot_info['shotHitBy']
        
        # Determine lastShotResult based on conditions
        if row_with_lastshot_info['isWinner'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Winner"
        elif row_with_lastshot_info['lastShotError'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Error"

In [1012]:
# # Display the modified DataFrame
# print(point_df[['rallyCount','lastShotDirection','lastShotFhBh', 'lastShotHitBy']])

In [1013]:
print(point_df.columns)

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'tiebreakScore', 'side', 'serverName', 'returnerName', 'Position',
       'pointEndPosition', 'Duration', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'isAce', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult'],
      dtype='object')


# pointWonBy

In [1014]:
# Initialize variables to keep track of the state
prev_point_number = None
point_won_by_list = []

# Iterate through the DataFrame
for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        # Check if pointNumber is different and consecutively increasing
        if prev_point_number is None or row['pointNumber'] == prev_point_number + 1:
            # Append pointWonBy to the list
            point_won_by_list.append(row['pointWonBy'])
            prev_point_number = row['pointNumber']
        else:
            print("Error: Point numbers are not different or consecutively increasing.")
            break

# Add point_won_by_list as a new column to point_df
point_df['pointWonBy'] = point_won_by_list

# Print point_df
print(point_df)


print(point_df['player1Name'].unique())
print(point_df['player2Name'].unique())
print(point_df['serverName'].unique())
print(point_df['returnerName'].unique())

# # Filter shot_data for rows where isPointEnd is 1
# filtered_df = shot_data[shot_data['isPointEnd'] == 1]

# # Record values for specified columns
# pointWonBy_df2 = filtered_df[['player1Name', 'player2Name', 'serverName', 'isWinner', 'pointNumber', 'shotInRally']].copy()

# # Determine returnerName based on serverName
# pointWonBy_df2['returnerName'] = pointWonBy_df2.apply(lambda row: row['player2Name'] if row['serverName'] == row['player1Name'] else row['player1Name'], axis=1)


# for index, row in pointWonBy_df2.iterrows():
#     if (row['isWinner'] == 1 and row['shotInRally'] % 2 == 1) or (row['isWinner'] != 1 and row['shotInRally'] % 2 == 0):
#         pointWonBy_df2.at[index, 'pointWonBy'] = row['serverName']
#     elif (row['isWinner'] == 1 and row['shotInRally'] % 2 == 0) or (row['isWinner'] != 1 and row['shotInRally'] % 2 == 1):
#         pointWonBy_df2.at[index, 'pointWonBy'] = row['returnerName']
#     else:
#         pointWonBy_df2.at[index, 'pointWonBy'] = None

# point_df['pointWonBy'] = pointWonBy_df2['pointWonBy'].values

     pointNumber    player1Name       player2Name pointScore gameScore  \
0              1  Tian Fang Ran  Sabina Zeynalova        0-0       0-0   
3              2  Tian Fang Ran  Sabina Zeynalova       0-15       0-0   
5              3  Tian Fang Ran  Sabina Zeynalova      15-15       0-0   
7              4  Tian Fang Ran  Sabina Zeynalova      30-15       0-0   
9              5  Tian Fang Ran  Sabina Zeynalova      40-15       0-0   
..           ...            ...               ...        ...       ...   
803          137  Tian Fang Ran  Sabina Zeynalova       15-0       5-4   
804          138  Tian Fang Ran  Sabina Zeynalova      15-15       5-4   
806          139  Tian Fang Ran  Sabina Zeynalova      30-15       5-4   
811          140  Tian Fang Ran  Sabina Zeynalova      30-30       5-4   
815          141  Tian Fang Ran  Sabina Zeynalova      40-30       5-4   

    tiebreakScore   side     serverName      returnerName   Position  ...  \
0            None  Deuce  Tian Fan

In [1015]:
# # Display the resulting DataFrame
print(point_df['pointWonBy'].unique())

['Sabina Zeynalova' 'Tian Fang Ran' None]


# Break Point

In [1016]:
# List of values to check for in 'Name' column
break_point_values = ['0-40', '15-40', '30-40', '40-40 (Deuce)', '40-40 (Ad)']

# Create 'isBreakPoint' column in point_df
point_df['isBreakPoint'] = point_df['pointScore'].isin(break_point_values)

In [1017]:
# Display the resulting DataFrame
# print(point_df[['pointNumber', 'Name', 'isBreakPoint']])

# Add Side

In [1018]:
# # List of values to check for in 'Name' column for Deuce
# deuce_values = ['0-0' ,'15-15','40-15','30-0' , '0-30','15-40' ,'40-40 (Deuce)','30-30']

# # Create 'servingSide' column in point_df
# point_df['servingSide'] = np.where(point_df['Name'].isin(deuce_values), 'Deuce', 'Ad')

In [1019]:
# # Display the resulting DataFrame
# print(point_df[['pointNumber', 'Name', 'servingSide']])

# At Net

- replace later with atNetAutomatically value

In [1020]:
import numpy as np

# Copy the DataFrame
at_net_df2 = shot_data.copy()

# Record values for specified columns
at_net_df2 = at_net_df2[['player1Name', 'player2Name', 'isVolley', 'serverName', 'shotHitBy', 'returnerName', 'pointNumber', 'shotInRally']].copy()

# Add columns for aggregation
at_net_df2['atNetPlayer1'] = np.where((at_net_df2['isVolley'] == 1) & (at_net_df2['shotHitBy'] == at_net_df2['player1Name']), 1, 0)
at_net_df2['atNetPlayer2'] = np.where((at_net_df2['isVolley'] == 1) & (at_net_df2['shotHitBy'] == at_net_df2['player2Name']), 1, 0)

# Aggregate atNetPlayer1 and atNetPlayer2 based on pointNumber
at_net_df2['atNetPlayer1_Agg'] = at_net_df2.groupby('pointNumber')['atNetPlayer1'].transform('max')
at_net_df2['atNetPlayer2_Agg'] = at_net_df2.groupby('pointNumber')['atNetPlayer2'].transform('max')

# Drop the duplicate rows created during aggregation
at_net_df2.drop_duplicates(subset=['pointNumber'], inplace=True)

point_df['atNetPlayer1'] = at_net_df2['atNetPlayer1_Agg'].values
point_df['atNetPlayer2'] = at_net_df2['atNetPlayer2_Agg'].values

In [1021]:
print(point_df.tail(10))

     pointNumber    player1Name       player2Name pointScore gameScore  \
762          132  Tian Fang Ran  Sabina Zeynalova       0-15       5-3   
770          133  Tian Fang Ran  Sabina Zeynalova      15-15       5-3   
774          134  Tian Fang Ran  Sabina Zeynalova      30-15       5-3   
789          135  Tian Fang Ran  Sabina Zeynalova      40-15       5-3   
791          136  Tian Fang Ran  Sabina Zeynalova        0-0       5-4   
803          137  Tian Fang Ran  Sabina Zeynalova       15-0       5-4   
804          138  Tian Fang Ran  Sabina Zeynalova      15-15       5-4   
806          139  Tian Fang Ran  Sabina Zeynalova      30-15       5-4   
811          140  Tian Fang Ran  Sabina Zeynalova      30-30       5-4   
815          141  Tian Fang Ran  Sabina Zeynalova      40-30       5-4   

    tiebreakScore   side        serverName      returnerName   Position  ...  \
762          None     Ad  Sabina Zeynalova     Tian Fang Ran  6943437.0  ...   
770          None  Deuce 

# Set Score
- Add Set Score
- Add setNum

In [1022]:
# Group shot_data by 'pointNumber' and get the first value of 'setScore'
setScore_df = shot_data.groupby('pointNumber')['setScore'].first().reset_index()

point_df['setScore'] = setScore_df['setScore'].values

In [1023]:
# Extract numbers from 'setScore' and calculate the sum plus 1
point_df['setNum'] = point_df['setScore'].apply(lambda x: sum(int(char) for char in x if char.isdigit()) + 1)

In [1024]:
# Create an empty DataFrame to store gameScore and setScore
setScore = shot_data.groupby('pointNumber').first()['setScore'].reset_index()
gameScore = shot_data.groupby('pointNumber').first()['gameScore'].reset_index()
pointScore = shot_data.groupby('pointNumber').first()['pointScore'].reset_index()
tiebreakScore = shot_data.groupby('pointNumber').first()['tiebreakScore'].reset_index()


point_df['setScore'] = setScore['setScore'].values
point_df['gameScore'] = gameScore['gameScore'].values
point_df['pointScore'] = pointScore['pointScore'].values
point_df['tiebreakScore'] = tiebreakScore['tiebreakScore'].values



In [1025]:
# Add Game Number, Set Number, Game/Set/Point for each player

point_df[['player1SetScore', 'player2SetScore']] = point_df['setScore'].str.split('-', expand=True)
point_df[['player1GameScore', 'player2GameScore']] = point_df['gameScore'].str.split('-', expand=True)
point_df[['player1PointScore', 'player2PointScore']] = point_df['pointScore'].str.split('-', expand=True)
point_df[['player1TiebreakScore', 'player2TiebreakScore']] = point_df['tiebreakScore'].str.split('-', expand=True)


def calculate_game_number(score):
    return int(score.split('-')[0]) + int(score.split('-')[1])+1

# Apply the function to create the 'gameNumber' column
point_df['gameNumber'] = point_df['gameScore'].apply(calculate_game_number)

In [1026]:
print(point_df['gameNumber'])

0       1
3       1
5       1
7       1
9       1
       ..
803    10
804    10
806    10
811    10
815    10
Name: gameNumber, Length: 141, dtype: int64


In [1027]:
print(point_df['serveResult'])

0      1st Serve In
3      Double Fault
5      Double Fault
7      Double Fault
9      Double Fault
           ...     
803    Double Fault
804    1st Serve In
806    1st Serve In
811    1st Serve In
815    1st Serve In
Name: serveResult, Length: 141, dtype: object


In [1028]:
# Add the 'player1ServeResult' column
point_df['player1ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServeResult'] = point_df['serveResult']

In [1029]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


In [1030]:
print(point_df['serveInPlacement'].unique())

['Wide' None 'T' 'Body' nan]


In [1031]:
# Add the 'player1ServePlacement' column
point_df['player1ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

In [1032]:
# Add the 'player1ReturnPlacement' column
point_df['player1ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnPlacement'] = point_df['returnDirection']

In [1033]:
# Add the 'player1ReturnFhBh' column
point_df['player1ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnFhBh'] = point_df['returnFhBh']

In [1034]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotPlacement'] = point_df['lastShotDirection']

In [1035]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotFhBh'] = point_df['lastShotFhBh']

In [1036]:
# Add the 'player1LastShotResult' column
point_df['player1LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player1Name']) & 
    ~point_df['player1ServeResult'].isin(['Ace', 'Double Fault']), 
    'player1LastShotResult'
] = point_df['lastShotResult']


# Add Name
- Changes PointScore
- Adds pointScoreCopy

In [1037]:
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore']])

     setNum gameScore pointScore tiebreakScore
0         1       0-0        0-0          None
3         1       0-0       0-15          None
5         1       0-0      15-15          None
7         1       0-0      30-15          None
9         1       0-0      40-15          None
..      ...       ...        ...           ...
803       2       5-4       15-0          None
804       2       5-4      15-15          None
806       2       5-4      30-15          None
811       2       5-4      30-30          None
815       2       5-4      40-30          None

[141 rows x 4 columns]


In [1038]:
# Change pointScore to the specified format
point_df['Name'] = point_df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)


In [1039]:
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']])

     setNum gameScore pointScore tiebreakScore  \
0         1       0-0        0-0          None   
3         1       0-0       0-15          None   
5         1       0-0      15-15          None   
7         1       0-0      30-15          None   
9         1       0-0      40-15          None   
..      ...       ...        ...           ...   
803       2       5-4       15-0          None   
804       2       5-4      15-15          None   
806       2       5-4      30-15          None   
811       2       5-4      30-30          None   
815       2       5-4      40-30          None   

                                        Name  
0      Set 1: 0-0, 0-0 Tian Fang Ran Serving  
3     Set 1: 0-0, 0-15 Tian Fang Ran Serving  
5    Set 1: 0-0, 15-15 Tian Fang Ran Serving  
7    Set 1: 0-0, 30-15 Tian Fang Ran Serving  
9    Set 1: 0-0, 40-15 Tian Fang Ran Serving  
..                                       ...  
803   Set 2: 5-4, 15-0 Tian Fang Ran Serving  
804  Set 2: 5-4, 15-15 

# EDA

In [1040]:
# read in csv here

In [1041]:
first_player1Name = point_df['player1Name'].iloc[0]



# Display the results
print(f"\nServe Results for {first_player1Name}:")

# Assuming point_df is your DataFrame
total_serves = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
first_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_count > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)])
second_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1)])
second_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("\nTotal Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df[point_df['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df['serverName'] == first_player1Name) & (point_df['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")

# Assuming point_df is your DataFrame
return_points = point_df[(point_df['returnerName'] == first_player1Name) & (point_df['rallyCount'] >= 2)]

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / total_return * 100 if total_return > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnCount * 100 if deuceReturnCount > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnCount * 100 if adReturnCount > 0 else 0




print(f"\nReturn Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)

# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Deuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("\nDeuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)

# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Ad Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("\nAd Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)

print(f"\nAt Net Results for {first_player1Name}:\n")


# Total points where atNetPlayer1 = 1
total_at_net_player1 = len(point_df[point_df['atNetPlayer1'] == 1])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df)) * 100 if len(point_df) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df[(point_df['atNetPlayer1'] == 1) & (point_df['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")








Serve Results for Tian Fang Ran:

Total Serves: 72
First Serve In (Count): 41
First Serve Won (Count): 20
First Serve In (%): 56.94%
First Serve Won (%): 48.78%
Second Serve In (Count): 17
Second Serve Total (Count): 31
Second Serve Won (Count): 9
Second Serve In (%): 54.84%
Second Serve Won (%): 52.94%
Ace (Count): 0
Double Fault (Count): 14
Points Won on Serve (Count) 35
Points Won on Serve (%): 48.61%

Return Results for Tian Fang Ran:

Total Return (Count): 68
Return Won (Count): 32
Return Won (%): 47.05882352941176

Return Made (Count): 51
Return Made (%): 0.75
Return Error (Count): 17
Return Winner (Count): 2

Deuce Return (Count): 36
Deuce Return Made (Count): 27
Deuce Return Made (%): 0.75
Deuce Return Won by Player1 (%): 44.44444444444444
Deuce Return Won by Player1 (Count): 16

Ad Return (Count): 32
Ad Return Made (Count): 24
Ad Return Made (%): 0.75
Ad Return Won by Player1 (Count): 18
Ad Return Won by Player1 (%): 56.25
Deuce Forehand Return Points - Made: 23
Deuce Forehan

In [1042]:
print(f"\nError Data for {first_player1Name}:\n")
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error')]

import numpy as np

# Filter rows without NaN values in relevant columns
forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = forehand_errors.shape[0]  # Count rows
backhand_counts = backhand_errors.shape[0]  # Count rows

# Print the total error counts for verification
total_error_counts = forehand_counts + backhand_counts

# Get value counts of 'errorType' for Forehand errors
forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print("Count of Backhand errors:", backhand_counts)
print("\nForehand errors:\n", forehand_error_types)
print("\nBackhand errors:\n", backhand_error_types)

print(forehand_errors[['lastShotDirection','pointScore', 'gameScore','setScore']])

print(backhand_errors[['lastShotDirection','pointScore', 'gameScore','setScore']])






# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)


# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)


# forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Forehand')]

# backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Backhand')]

# # Count the occurrences of 'Forehand' and 'Backhand' separately
# forehand_counts = len(forehand_errors)
# backhand_counts = len(backhand_errors)

# # Print the total error counts for verification
# total_error_counts = len(forehand_errors) + len(backhand_errors)

# # Get value counts of 'errorType' for Forehand errors
# forehand_error_types = forehand_errors['errorType'].value_counts()

# # Get value counts of 'errorType' for Backhand errors
# backhand_error_types = backhand_errors['errorType'].value_counts()

# # Print the counts and error types
# print("Count of Total errors:", total_error_counts)
# print("Count of Forehand errors:", forehand_counts)
# print("Count of Backhand errors:", backhand_counts)
# print("\nForehand errors:\n", forehand_error_types)
# print("\nBackhand errors:\n", backhand_error_types)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
# forehand_error_counts = forehand_errors.groupby(['lastShotDirection', 'errorType']).size()

# print("Value counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
# backhand_error_counts = backhand_errors.groupby(['lastShotDirection', 'errorType']).size()


# print("Value counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)


# forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Forehand')]

# backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Backhand')]

# # Count the occurrences of 'Forehand' and 'Backhand' separately
# forehand_counts = len(forehand_errors)
# backhand_counts = len(backhand_errors)
# total_error_counts = len(total_errors)

# # Get value counts of 'errorType' for Forehand errors
# forehand_error_types = forehand_errors['errorType'].value_counts()

# # Get value counts of 'errorType' for Backhand errors
# backhand_error_types = backhand_errors['errorType'].value_counts()
# print("Count of Total errors:", total_error_counts)
# print("Count of Forehand errors:", forehand_counts)
# print("Count of Backhand errors:", backhand_counts)
# print("\nForehand errors:\n", forehand_error_types)
# print("\nBackhand errors:\n", backhand_error_types)



#


Error Data for Tian Fang Ran:

Count of Total errors: 49
Count of Forehand errors: 30
Count of Backhand errors: 19

Forehand errors:
 errorType
Long          14
Net           10
Wide Left      4
Wide Right     2
Name: count, dtype: int64

Backhand errors:
 errorType
Net           7
Long          5
Wide Right    4
Wide Left     3
Name: count, dtype: int64
    lastShotDirection pointScore gameScore setScore
0          Crosscourt        0-0       0-0      0-0
9       Down the Line      40-15       0-0      0-0
22      Down the Line        0-0       1-0      0-0
44      Down the Line      15-30       1-0      0-0
67         Crosscourt      15-40       2-0      0-0
70         Crosscourt        0-0       2-1      0-0
84         Crosscourt       0-15       2-1      0-0
125        Crosscourt       0-15       3-1      0-0
179     Down the Line      30-30       3-2      0-0
183        Crosscourt      40-30       3-2      0-0
257        Crosscourt        0-0       4-3      0-0
287        Crossco

In [1043]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'tiebreakScore', 'side', 'serverName', 'returnerName', 'Position',
       'pointEndPosition', 'Duration', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'isAce', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult', 'pointWonBy',
       'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'setScore', 'setNum',
       'player1SetScore', 'player2SetScore', 'player1GameScore',
       'player2GameScore', 'player1PointScore', 'player2PointScore',
       'player1TiebreakScore', 'player2TiebreakScore', 'gameNumber',
       'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
       'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
       'player1LastShotResult', 'Name'],
      d

# Continue down for print to Visuals and print to Json

In [1044]:
first_player1Name = point_df['player1Name'].iloc[0]
print(first_player1Name)

Tian Fang Ran


In [1045]:
# Assuming point_df is your DataFrame
total_count = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
percentage_first_serve_in = (first_serve_in_count / total_count) * 100 if total_count > 0 else 0

# Display the results
print("Total Count:", total_count)
print("Count of First Serve In:", first_serve_in_count)
print(f"First Serve In Percentage: {percentage_first_serve_in:.2f}%")


Total Count: 72
Count of First Serve In: 41
First Serve In Percentage: 56.94%


In [1046]:
# Assuming point_df is your DataFrame
total_serves = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
first_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_count > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)])
second_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1)])
second_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("Total Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df[point_df['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df['serverName'] == first_player1Name) & (point_df['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")


Total Serves: 72
First Serve In (Count): 41
First Serve Won (Count): 20
First Serve In (%): 56.94%
First Serve Won (%): 48.78%
Second Serve In (Count): 17
Second Serve Total (Count): 31
Second Serve Won (Count): 9
Second Serve In (%): 54.84%
Second Serve Won (%): 52.94%
Ace (Count): 0
Double Fault (Count): 14
Points Won on Serve (Count) 35
Points Won on Serve (%): 48.61%


In [1047]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'tiebreakScore', 'side', 'serverName', 'returnerName', 'Position',
       'pointEndPosition', 'Duration', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'isAce', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult', 'pointWonBy',
       'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'setScore', 'setNum',
       'player1SetScore', 'player2SetScore', 'player1GameScore',
       'player2GameScore', 'player1PointScore', 'player2PointScore',
       'player1TiebreakScore', 'player2TiebreakScore', 'gameNumber',
       'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
       'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
       'player1LastShotResult', 'Name'],
      d

In [1048]:
point_df['lastShotResult'].unique()

array(['Error', 'Winner', None], dtype=object)

In [1049]:
# Assuming point_df is your DataFrame
return_points = point_df[(point_df['returnerName'] == first_player1Name) & (point_df['rallyCount'] >= 2)]

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / total_return * 100 if total_return > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnCount * 100 if deuceReturnCount > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnCount * 100 if adReturnCount > 0 else 0




print(f"Return Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)


Return Results for Tian Fang Ran:

Total Return (Count): 68
Return Won (Count): 32
Return Won (%): 47.05882352941176

Return Made (Count): 51
Return Made (%): 0.75
Return Error (Count): 17
Return Winner (Count): 2

Deuce Return (Count): 36
Deuce Return Made (Count): 27
Deuce Return Made (%): 0.75
Deuce Return Won by Player1 (%): 44.44444444444444
Deuce Return Won by Player1 (Count): 16

Ad Return (Count): 32
Ad Return Made (Count): 24
Ad Return Made (%): 0.75
Ad Return Won by Player1 (Count): 18
Ad Return Won by Player1 (%): 56.25


In [1050]:
# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Deuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("\nDeuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)


Deuce Forehand Return Points - Made: 23
Deuce Forehand Return Points - Error: 7

Deuce Backhand Return Points - Made: 4
Deuce Backhand Return Points - Error: 2


In [1051]:
# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Ad Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("\nAd Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)


Ad Forehand Return Points - Made: 12
Ad Forehand Return Points - Error: 5

Ad Backhand Return Points - Made: 12
Ad Backhand Return Points - Error: 3


In [1052]:
# Total points where atNetPlayer1 = 1
total_at_net_player1 = len(point_df[point_df['atNetPlayer1'] == 1])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df)) * 100 if len(point_df) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df[(point_df['atNetPlayer1'] == 1) & (point_df['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")


Total Net Points for Tian Fang Ran: 6
Percentage of Net Points for Tian Fang Ran: 4.26%

Total Net Points won by Tian Fang Ran: 4
Percentage of Net Points won by Tian Fang Ran: 66.67%


In [1053]:
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error')]

forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand')]

backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand')]

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = len(forehand_errors)
backhand_counts = len(backhand_errors)

# Print the total error counts for verification
total_error_counts = len(forehand_errors) + len(backhand_errors)

# Get value counts of 'errorType' for Forehand errors
forehand_error_types = forehand_errors['errorType'].value_counts()

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts()

# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print("Count of Backhand errors:", backhand_counts)
print("\nForehand errors:\n", forehand_error_types)
print("\nBackhand errors:\n", backhand_error_types)

# forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Forehand')]

# backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Backhand')]

# # Count the occurrences of 'Forehand' and 'Backhand' separately
# forehand_counts = len(forehand_errors)
# backhand_counts = len(backhand_errors)
# total_error_counts = len(total_errors)

# # Get value counts of 'errorType' for Forehand errors
# forehand_error_types = forehand_errors['errorType'].value_counts()

# # Get value counts of 'errorType' for Backhand errors
# backhand_error_types = backhand_errors['errorType'].value_counts()
# print("Count of Total errors:", total_error_counts)
# print("Count of Forehand errors:", forehand_counts)
# print("Count of Backhand errors:", backhand_counts)
# print("\nForehand errors:\n", forehand_error_types)
# print("\nBackhand errors:\n", backhand_error_types)


Count of Total errors: 49
Count of Forehand errors: 30
Count of Backhand errors: 19

Forehand errors:
 errorType
Long          14
Net           10
Wide Left      4
Wide Right     2
Name: count, dtype: int64

Backhand errors:
 errorType
Net           7
Long          5
Wide Right    4
Wide Left     3
Name: count, dtype: int64


In [1054]:
# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['lastShotDirection', 'errorType']).size()

print("Value counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)

Value counts of 'errorType' for Forehand errors with different directions:
 lastShotDirection  errorType 
Crosscourt         Long          11
                   Net            6
                   Wide Left      2
                   Wide Right     1
Down the Line      Long           3
                   Net            4
                   Wide Left      2
                   Wide Right     1
dtype: int64


In [1055]:

# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['lastShotDirection', 'errorType']).size()

print("Value counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)

Value counts of 'errorType' for Backhand errors with different directions:
 lastShotDirection  errorType 
Crosscourt         Long          1
                   Net           2
                   Wide Left     2
                   Wide Right    4
Down the Line      Long          4
                   Net           5
                   Wide Left     1
dtype: int64


In [1056]:
print(point_df['errorType'].unique())

['Net' 'Long' None 'Wide Left' 'Wide Right']


# PRINT POINTCSV VISUALS

In [1057]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = point_df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = point_df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df.to_csv(f'{player1NameNoSpace}_{player2NameNoSpace}_Point_ForVisuals.csv', index=False)


# Remove points

In [1058]:
# Assuming pointdf is a DataFrame

# Display rows where 'isPointStart' is equal to 1 and 'pointStartTime' is NaN in pointdf
missing_start_time = point_df[point_df['Position'].isna() | (point_df['Position'] == "")]
print("Rows with missing start time:")
print(missing_start_time)



Rows with missing start time:
     pointNumber    player1Name       player2Name pointScore gameScore  \
586           92  Tian Fang Ran  Sabina Zeynalova      40-15       1-0   
616          104  Tian Fang Ran  Sabina Zeynalova        0-0       3-1   
774          134  Tian Fang Ran  Sabina Zeynalova      30-15       5-3   

    tiebreakScore   side        serverName      returnerName Position  ...  \
586          None  Deuce     Tian Fang Ran  Sabina Zeynalova     None  ...   
616          None  Deuce  Sabina Zeynalova     Tian Fang Ran     None  ...   
774          None     Ad  Sabina Zeynalova     Tian Fang Ran     None  ...   

    player2TiebreakScore gameNumber  player1ServeResult player1ServePlacement  \
586                 None          2        1st Serve In              Deuce: T   
616                 None          5                None                  None   
774                 None          9                None                  None   

     player1ReturnPlacement  player

In [1059]:

# Remove those rows from pointdf
point_df_for_viewer = point_df.drop(missing_start_time.index)

In [1060]:
# Change Value Names for Match Viewer Output

# Assuming point_df is your DataFrame and player1Name is the name from the first row
player1Name = point_df_for_viewer.iloc[0]['player1Name']

# Replace values in the 'atNetPlayer1' column
point_df_for_viewer['atNetPlayer1'] = point_df_for_viewer['atNetPlayer1'].replace({0: "", 1: player1Name})

In [1061]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
# Replace NaN, None, and NA values with empty strings in point_df
point_df_for_viewer.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# Print POINTCSV UPLOAD

In [1062]:
player1Name = point_df_for_viewer.iloc[0]['player1Name'].replace(" ", "")
player2Name = point_df_for_viewer.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df_for_viewer.to_csv(f'{player1Name}_{player2Name}_Point_ForUpload.csv', index=False)
point_df_for_viewer.to_json(f'{player1Name}_{player2Name}_Point_ForUpload.json', orient='records')

In [1063]:
# git fetch origin
# git pull
# git checkoutbranchName
# git add .
# git commit -m "message"
# git push